## Object Interception Using Inverse Kinematics

### Imports

In [1]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os
import random
import pydot
from pydrake.all import (LeafSystem,Sphere, Box )

from manipulation import running_as_notebook
import matplotlib.pyplot as plt
from IPython.display import HTML, SVG, display

from pydrake.common import FindResourceOrThrow, temp_directory
from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Role,
    StartMeshcat,
)


from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.meshcat import JointSliders
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation.scenarios import MakeManipulationStation


from pydrake.symbolic import Variable
from pydrake.systems.primitives import SymbolicVectorSystem
from pydrake.systems.framework import LeafSystem

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad
from pydrake.all import (ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant, Parser,
                         PiecewisePolynomial, PiecewiseQuaternionSlerp,
                         RigidTransform, RollPitchYaw, RotationMatrix,
                         Simulator, Solve, StartMeshcat, TrajectorySource)
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial
from pydrake.trajectories import PiecewisePolynomial


### Mesh-cat Loading

In [2]:
# Create a Drake temporary directory to store files.
# Note: this tutorial will create two temporary files (cylinder.sdf and
# table_top.sdf) under `/tmp/robotlocomotion_drake_xxxxxx` directory.
temp_dir = temp_directory()

# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

#meshcat.StaticHtml()

INFO:drake:Meshcat listening for connections at https://83195575-1d79-49de-b013-31de4398d0b7.deepnoteproject.com/7001/


### SDF Model inspector Function

In [3]:
def model_inspector(filename):
    meshcat.Delete()
    meshcat.DeleteAddedControls()
    builder = DiagramBuilder()

    # Note: the time_step here is chosen arbitrarily.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    # Load the file into the plant/scene_graph.
    parser = Parser(plant)
    parser.AddModelFromFile(filename)
    plant.Finalize()

    # Add two visualizers, one to publish the "visual" geometry, and one to
    # publish the "collision" geometry.
    visual = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kPerception, prefix="visual"))
    collision = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(role=Role.kProximity, prefix="collision"))
    # Disable the collision geometry at the start; it can be enabled by the
    # checkbox in the meshcat controls.
    meshcat.SetProperty("collision", "visible", False)

    # Set the timeout to a small number in test mode. Otherwise, JointSliders
    # will run until "Stop JointSliders" button is clicked.
    default_interactive_timeout = 1.0 if "TEST_SRCDIR" in os.environ else None
    sliders = builder.AddSystem(JointSliders(meshcat, plant))
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)

### SDF for Different Bodies                                                                                                                                                                                                                               

In [4]:
### Project XML file under Models Folder  
project_xml_file = os.path.join("/work/Models", "project.xml")
project_xml = """<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
"""

with open(project_xml_file, "w") as f:
    f.write(project_xml)


### GOALIE END-EFFECTOR
goalie_end_sdf_file = os.path.join("/work/Models", "goalie_end.sdf")
goalie_end_sdf = """<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
<sdf version="1.7">
  <model name="goalie_end">
    <link name="goalie_end_link">
    
      <visual name="visual">
        <pose>0 0 0.15 0 0 0</pose>
        <geometry>
          <box>
            <size>.30 .06 0.30</size>
          </box>
        </geometry>
       
      </visual>
      
      <collision name="collision">
        <pose>0 0 .15 0 0 0</pose>
        <geometry>
          <box>
            <size>.30 .06 0.30</size>
          </box>
        </geometry>
      </collision>
    </link>
    <frame name="goalie_end_center">
      <pose relative_to="goalie_end_link">0 0 0.15 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(goalie_end_sdf_file, "w") as f:
    f.write(goalie_end_sdf)


### GOAL POST:
goalpost_sdf_file = os.path.join("/work/Models", "goalpost.sdf")
goalpost_sdf = """
<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
<sdf version="1.7">
  <model name="goalpost">

    <link name="goalpost_link">

      <visual name="visual">
        <pose>0 0 0 0 0 3.1415693</pose>
        <geometry>
            <mesh>
                <uri>/work/soccerpostfixed.obj</uri>
                <scale> 0.02 0.02 0.02 </scale>
            </mesh>
        </geometry>
      </visual>

      <collision name="collision">
         <pose>0.05  -0.4 0.6 0 0 0</pose>
        <geometry>
            <box>
            <size>2 0.1 1.2</size>
          </box>
        </geometry>
      </collision>

    </link>

    <frame name="goalpost_center">
      <pose relative_to ="goalpost_link">0 0 0.50 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(goalpost_sdf_file, "w") as f:
    f.write(goalpost_sdf)



### Floor:
floor_sdf_file = os.path.join("/work/Models", "floor.sdf")
floor_sdf = """
<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
<sdf version="1.7">
  <model name="floor">

    <link name="floor_link">

      <visual name="visual">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
            <box>
                <size>10 20 0.1</size>
            </box>
        </geometry>
      </visual>

      <collision name="collision">
         <pose>0 0 0 0 0 0</pose>
        <geometry>
            <box>
            <size>10 20 0.1</size>
          </box>
        </geometry>
      </collision>

    </link>

    <frame name="floor_center">
      <pose relative_to ="floor_link">0 0 .05 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(floor_sdf_file, "w") as f:
    f.write(floor_sdf)


### SOCCER BALL
Soccer_Ball_sdf_file = os.path.join("/work/Models", "Ball.sdf")
Soccer_Ball_sdf = """<?xml version="1.0"?>
<package>
  <name>project</name>
</package>
<sdf xmlns:xacro="http://www.ros.org/wiki/xacro" version="1.7">
  <model name="Soccer Ball">
    <link name="Soccer_Ball_link">

      <inertial>
        <pose>0 0 0.15 0 0 0</pose>
        <mass>0.5</mass>
        <inertia>
          <ixx>0.008</ixx> 
          <iyy>0.008</iyy>
          <izz>0.008</izz>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyz>0</iyz>
        </inertia>
      </inertial>


      <visual name="visual">
        <pose>0 0 0.15 0 0 0</pose>
        <geometry>
          <mesh>
            <uri>/work/Ball.obj</uri>
            <scale> .1 .1 .1 </scale>
          </mesh>
        </geometry>

        <material>
         <diffuse>0.9 0.8 0.7 1.0</diffuse>
        </material>
      </visual>


      <collision name="collision">
        <pose>0 0 .15 0 0 0</pose>
        <geometry>
          <sphere>
            <radius>.1</radius>
          </sphere>
        </geometry>


        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            10000
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            0.5
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>


      </collision>
    </link>
    <frame name="Soccer_ball_center">
      <pose relative_to="Soccer_Ball_link">0 0 0 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

with open(Soccer_Ball_sdf_file, "w") as f:
    f.write(Soccer_Ball_sdf)
    

### Model Directives

In [5]:
model_directives = """
    directives:
    - add_frame:
        name: scene 
        X_PF:
            base_frame: world
            rotation: !Rpy { deg: [-90.0, 0.0, 0 ]}
            translation: [5,5,5]

    - add_model:
        name: iiwa
        file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
        default_joint_positions:
            iiwa_joint_1: [-1.5]
            iiwa_joint_2: [0]
            iiwa_joint_3: [1.5]
            iiwa_joint_4: [-1.5]
            iiwa_joint_5: [0]
            iiwa_joint_6: [ 1.65]
            iiwa_joint_7: [1.65]
    - add_weld:
        parent: world
        child: iiwa::iiwa_link_0
        X_PC:
            translation: [0, 0, 0]
            rotation: !Rpy { deg: [0, 0, 0]}
    - add_model:
        name: floor
        file: package://project/floor.sdf
    - add_weld:
        parent: world
        child: floor_center
        X_PC:
            translation: [0, 0, 0]
            rotation: !Rpy { deg: [0, 0, 0]}
    - add_model:
        name: goalie_end
        file: package://project/goalie_end.sdf
    - add_weld:
        parent: iiwa::iiwa_link_7
        child: goalie_end::goalie_end_link
        X_PC:
            translation: [0, 0, .04]
            rotation: !Rpy { deg: [0, 0, 90]}
    - add_model:
        name: goalpost
        file: package://project/goalpost.sdf
    - add_weld:
        parent: floor_center
        child: goalpost_center
        X_PC:
            translation: [0, -0.8, 0.5]
            rotation: !Rpy { deg: [0, 0, 0]}
    - add_model:
        name: Soccer_Ball
        file: package://project/Ball.sdf
        default_free_body_pose:
            Soccer_Ball_link:
                translation: [0, 5, 0]


    """

### Simulation:

#### Soccer Ball Trajectory Setup:

- Assume no drag
- Goal post "virtual wall" in x-z plane 
- Initial pose of ball =  $[0, y_0, 0] $ 
- Desired pose of ball = $ [x_f, 0, z_f]$, where $x_f \in [-0.85, 0.85] $ and $z_f \in [.2, 0.90]$ 



<div id="image-table">
    <table>
	    <tr>
            <td style="padding:10px">
            	<img src="Image/image-20221207-005332.png" width="425"/>
            </td>
    	    <td style="padding:10px">
        	    <img src="Image/image-20221207-005953.png" width="400"/>
      	    </td>
        </tr>
    </table>
</div>





In [6]:
initial_soccer_params = {
    "s_ball_goalpost": 10,
    "desired_approach_speed": np.random.uniform(5,10), 
    "x_range": 0.85,
    "z_range": 0.90
}


Given goal post bound, distance of soccer and randomized approach speed and desired final pose when it hits the goal post , use free body kinematics to calculate projectile  speed

In [7]:
def get_desired_velocity(desired_pos, params = initial_soccer_params, verbose=False):
    """
    Given final x and z position of the ball when it enters the goal post, 
    returns Velocity vector with which ball should be launched from the initial position 
    """
    s,vy_des  = params["s_ball_goalpost"],params["desired_approach_speed"]
    x_des,_,z_des = desired_pos

    tau = s/vy_des #Time for ball to reach the goal post 
    vx_des = x_des/tau #No external force in x-direction
    vz_des = (z_des + 1/2*9.81*tau**2)/tau
    if verbose: print("Projectile Time (Tau): ", tau, "s")
    return (vx_des, -vy_des, vz_des) # vy = -vy because of the way we are simulating the env.  
    

def randomize_ball_trajectory(params = initial_soccer_params, verbose=False):
    '''
    Randomize the ball trajectory with boundary given by initial_soccer_params
    '''
    x_rand = random.uniform(-params["x_range"], params["x_range"])
    z_rand = random.uniform(.02, params["z_range"])
    desired_pos = (x_rand,0.0, z_rand)
    if verbose: print("desired coordinate in goal post:", desired_pos)

    v = get_desired_velocity(desired_pos, verbose=verbose)
    if verbose: print("Projectile initial speed: ", v)
    return v



def randomize_ball_trajectory_w_dp(params = initial_soccer_params, verbose=False):
    '''
    Randomize the ball trajectory with boundary given by initial_soccer_params and returns
    calculated desired/final postition for ball
    '''
    x_rand = random.uniform(-params["x_range"], params["x_range"])
    z_rand = random.uniform(.2, params["z_range"])
    desired_pos = (x_rand,0.0, z_rand)
    if verbose: print("desired coordinate in goal post:", desired_pos)

    v = get_desired_velocity(desired_pos, verbose=verbose)
    if verbose: print("Projectile initial speed: ", v)
    return v, desired_pos

randomize_ball_trajectory_w_dp(verbose = True)


desired coordinate in goal post: (-0.2083693650010443, 0.0, 0.24652326898888377)
Projectile Time (Tau):  1.383718102032297 s
Projectile initial speed:  (-0.15058657156758132, -7.226905527442895, 6.965297327998324)


((-0.15058657156758132, -7.226905527442895, 6.965297327998324),
 (-0.2083693650010443, 0.0, 0.24652326898888377))

## Base Manipulator

As a workaround to having to weld the created end-effector to the controller plant. We will be manipulating the Iiwa with reference to its final link. The offsets necessary for the end effector to hit the ball will be handled when the trajectory poses are calculated

In [8]:
def create_q_knots(pose_lst, controller_plant, link_frame):
  """Convert end-effector pose list to joint position list using series of
  InverseKinematics problems. Note that q is 7-dimensional no wsg gripper in controller controller_plant.
  @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
  @param: controller_plant: The controller plant which contains the iiwa robot in our simulation.
  @param: link_frame: frame of reference with respect to world of the iiwa_link_7.
  @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
  """
  q_knots = []
  world_frame = controller_plant.world_frame()
  
  q_nominal = np.array([ -1.5, 0.0, 1.5, -1.5, 0, 1.65, 1.65]) # nominal joint for joint-centering.

  # Change this so it's okay to have no contrain on y
  def AddOrientationConstraint(ik, R_WG, bounds):
    """Add orientation constraint to the ik problem. Implements an inequality
    constraint where the axis-angle difference between f_R(q) and R_WG must be
    within bounds. Can be translated to:
    ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
    """
    ik.AddOrientationConstraint(
        frameAbar=world_frame, R_AbarA=R_WG,
        frameBbar=link_frame, R_BbarB=RotationMatrix(),
        theta_bound=bounds
    )

  def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
    """Add position constraint to the ik problem. Implements an inequality
    constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
    translated to
    ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
    """
    ik.AddPositionConstraint(
        frameA=world_frame, frameB=link_frame, p_BQ=np.array([0, 0, 0]),
        p_AQ_lower=p_WG_lower, p_AQ_upper=p_WG_upper)

  for i in range(len(pose_lst)):
    ik = inverse_kinematics.InverseKinematics(controller_plant)
    q_variables = ik.q() # Get variables for MathematicalProgram
    prog = ik.prog() # Get MathematicalProgram

    #### Modify here ###############################

    pos = pose_lst[i].translation()

    rot = pose_lst[i].rotation()

    # Adds orientation constraint so that end_effector hits ball with flat surface
    AddOrientationConstraint(ik, RotationMatrix(rot), np.pi/4)

    # Adds position constraint so that the iiwa_link_7 follows passed through pose_lst
    AddPositionConstraint(ik, pos - [.009, .01, .009], pos + [.009, .01, .009])

    prog.AddQuadraticErrorCost(np.identity(len(q_variables)), q_nominal, q_variables)

    if i == 0:
        prog.SetInitialGuess(q_variables, q_nominal)
    else:
        prog.SetInitialGuess(q_variables, q_knots[-1])
    ################################################
  
    result = Solve(prog)

    if not result.is_success():
      raise RuntimeError
      
    q_knots.append(result.GetSolution(q_variables))
    
  return q_knots

Functions for simulation building and running

In [9]:
params = initial_soccer_params

def setup_manipulation_station():
  builder = DiagramBuilder()
  station = builder.AddSystem(
        MakeManipulationStation(
            model_directives, package_xmls=["/work/Models/project.xml"], time_step = 1e-3
            )
        )
  plant = station.GetSubsystemByName("plant")
  controller_plant = station.GetSubsystemByName(
      "iiwa_controller").get_multibody_plant_for_control()
  scene_graph = station.GetSubsystemByName('scene_graph')
  AddMultibodyTriad(
      plant.GetFrameByName("goalie_end_link"), scene_graph)

  MeshcatVisualizer.AddToBuilder(
        builder,
        station.GetOutputPort("query_object"),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False))

  diagram = builder.Build()

  context = plant.CreateDefaultContext()
  end_effector = plant.GetBodyByName("goalie_end_link")

  initial_pose = plant.EvalBodyPoseInWorld(context, end_effector)

  simulator = Simulator(diagram)
  simulator.set_target_realtime_rate(1.0)
  simulator.AdvanceTo(0.01)

  return initial_pose

# Get initial pose of the gripper by using default context of manip station.
initial_pose = setup_manipulation_station()

def CreateIiwaControllerPlant():
    """creates plant that includes only the robot and end_effector, used for controller."""
    robot_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
    end_effector_sdf_path = "/work/Models/goalie_end.sdf"
    sim_timestep = 1e-3
    plant_robot = MultibodyPlant(sim_timestep)
    parser = Parser(plant=plant_robot)
    parser.AddModelFromFile(robot_sdf_path)
    parser.AddModelFromFile(end_effector_sdf_path)
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.world_frame(),
        frame_on_child_M=plant_robot.GetFrameByName("iiwa_link_0"))
    plant_robot.WeldFrames(
        frame_on_parent_F=plant_robot.GetFrameByName("iiwa_link_7"),
        frame_on_child_M=plant_robot.GetFrameByName("goalie_end_link"),
        X_FM=RigidTransform(RollPitchYaw(0, 0, 0), np.array([0, 0, .04]))
    )
    plant_robot.Finalize()

    link_frame_indices = []
    for i in range(8):
        link_frame_indices.append(
            plant_robot.GetFrameByName("iiwa_link_" + str(i)).index())

    return plant_robot, link_frame_indices

# Leaf system for following trajectory. 
# Necessary to be able to input trajectory after the builder has been built
class MyTrajectorySource(LeafSystem):
    def __init__(self, trajectory):
        LeafSystem.__init__(self)
        self._trajectory = trajectory

        self.DeclareVectorOutputPort(
            "value", trajectory.rows(),
            self.CalcOutput)

    def CalcOutput(self, context, output):
        output.SetFromVector(self._trajectory.value(context.get_time()))


# Main function that runs and simulates the inverse kinematics object interceptor
def BuildAndSimulateTrajectory():
  """Simulate trajectory for manipulation station.
  """
  # Clear meshcat
  meshcat.Delete()
  meshcat.DeleteAddedControls()

  # Set up simulation
  builder = DiagramBuilder()
  station = builder.AddSystem(
        MakeManipulationStation(
            model_directives, package_xmls=["/work/Models/project.xml"], time_step = 1e-3
            )
        )
  
  plant = station.GetSubsystemByName("plant")
  controller_plant = station.GetSubsystemByName(
      "iiwa_controller").get_multibody_plant_for_control() #Note only includes the iiwa robot
  
  scene_graph = station.GetSubsystemByName('scene_graph')
  
  # Add meshcat visualizer.
  visualizer = MeshcatVisualizer.AddToBuilder(
      builder, station.GetOutputPort("query_object"), meshcat)

  # Add a triad on the iiwa_link_7 to be able to compare with the trajectory calculated
  AddMultibodyTriad(
      plant.GetFrameByName("iiwa_link_7"), scene_graph)

  # Make trajectory system using leaf system created (MyTrajectorySource) which can be updated after
  # the builder is built and the trajectory poses are calculated
  q_traj_system = builder.AddSystem(MyTrajectorySource(PiecewisePolynomial(np.zeros((7,1)))))

  # Connect the trajectory system
  builder.Connect(q_traj_system.get_output_port(),
                  station.GetInputPort("iiwa_position"))

  # Finish drake plant set up
  diagram = builder.Build()
  simulator = Simulator(diagram)
  context = simulator.get_mutable_context()
  plant_context = plant.GetMyMutableContextFromRoot(context)
  SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
  station_context = station.GetMyMutableContextFromRoot(context)

  ### Soccer Ball Trajectory Randomization Basic Setup:
  #As of now, soccer ball is ModelInstanceIndex(5)
  soccer_ball_model = plant.GetModelInstanceByName('Soccer_Ball')

  v, desired_pos = randomize_ball_trajectory_w_dp()
  vx, vy, vz = v
  xf, yf, zf = desired_pos

  # Calculate initial pose and intersection pose
  end_effector = plant.GetBodyByName('iiwa_link_7') # Note that our end-effector is the iiwa_link_7
  initial_pose = end_effector.EvalPoseInWorld(plant_context)
  initial_rotation = initial_pose.rotation()
  # Here we offset the final intersection pose to sligthly infront of the robot and have the trajectory be
  # slightly above where the ball will hit so that the goalie end effector will be what should make contact
  # with the ball. infront = .35 m and offset for end_effector = .35 m
  intersection_pose = RigidTransform(initial_rotation, [xf, yf+.35, zf+.35])

  # Set the initial positions and velocities of the ball from randomized trajectory
  plant.SetPositions(plant_context,soccer_ball_model, np.array([1,0,0,0,0,params["s_ball_goalpost"]+0.35, 0]))
  plant.SetVelocities(plant_context,soccer_ball_model, np.array([0,0,0,vx,vy,vz]))

  iiwa_model = plant.GetModelInstanceByName('iiwa')

  ### Interpolate Pose for interception.
  def make_orientation_trajectory():
    traj = PiecewiseQuaternionSlerp()
    traj.Append(0.0, initial_pose.rotation())
    traj.Append(1.0, intersection_pose.rotation())
    return traj

  def make_position_trajectory():
    traj = PiecewisePolynomial.FirstOrderHold(
        [0.0, 1.0],
        np.vstack([[initial_pose.translation()],
                    [intersection_pose.translation()]]).T)
    return traj

  traj_rotation = make_orientation_trajectory()
  traj_translation = make_position_trajectory()

  def InterpolatePose(t):
    return RigidTransform(RotationMatrix(traj_rotation.orientation(t)),
                            traj_translation.value(t))

  # Visualize our end-effector nominal trajectory.
  t_lst = np.linspace(0, 2, 30)
  pose_lst = []
  for t in t_lst:
      AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePose(t), opacity=.2)
      pose_lst.append(InterpolatePose(t))

  # Get solution to ik problem
  q_knots = np.array(create_q_knots(pose_lst, controller_plant, controller_plant.GetFrameByName("iiwa_link_7")))
  
  #q_knots = np.array(create_q_knots(pose_lst, controller_plant, plant.GetFrameByName("goalie_end_link")))

  q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)

  # Pass solution into trajectory system
  q_traj_system._trajectory = q_traj

  # Visualize and return html file
  visualizer.StartRecording(False)
  simulator.AdvanceTo(q_traj.end_time())
  visualizer.PublishRecording()
  html_file_path = os.path.join("/work/Models", "ex1.html")
  html_file = meshcat.StaticHtml()
  with open(html_file_path, "w") as f:
    f.write(html_file)
  
  return simulator

#   if running_as_notebook:
#         simulator.set_target_realtime_rate(1.0)
#         visualizer.StartRecording(True)
#         meshcat.AddButton("Stop Simulation", "Escape")
#         print("Press Escape to stop the simulation")
#         while meshcat.GetButtonClicks("Stop Simulation") < 1:
#             simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
#         visualizer.PublishRecording()
#         meshcat.DeleteButton("Stop Simulation")
#         print("Simulation complete")

#   return simulator

## Run simulation

The below code will try to run a simulation. Due to IK constraints and runtime sometimes solutions are not found so the below code will loop until a trajectory is found for the robot.

In [1]:
solved = False
while not solved:
    try:
        simulator = BuildAndSimulateTrajectory()
        solved = True
    except:
        print("IK unsolvable")

IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK unsolvable
IK uns

KeyboardInterrupt: 

In [12]:
# Using MakeManipulationStation function
def create_soccer_scene_base(sim_time_step = 1e-3, params = initial_soccer_params, teleop = False):

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            model_directives, package_xmls=["/work/Models/project.xml"], time_step = 0.001
            )
        )
        
    plant = station.GetSubsystemByName("plant")
    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    
    # Add a meshcat visualizer.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)

    if teleop:
        #Add Joint Slider Element
        default_interactive_timeout = 1.0 if "TEST_SRCDIR" in os.environ else None
        sliders = builder.AddSystem(JointSliders(meshcat, plant))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())
    station_context = station.GetMyMutableContextFromRoot(context)

    plant_context = plant.GetMyMutableContextFromRoot(context)

    #Set Iiwa to Default joint position
    q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('iiwa'))
    station.GetInputPort('iiwa_position').FixValue(station_context, q0)

    #Soccer Ball Trajectory Randomization Basic Setup:
    #As of now, soccer ball is ModelInstanceIndex(5)
    soccer_ball_model = plant.GetModelInstanceByName('Soccer_Ball')
    # current_pose = plant.GetPositions(plant_context, soccer_ball_model) # 7 elements: 4 Quaternions, 3 positions
    # current_speed = plant.GetVelocities(plant_context, soccer_ball_model) #6 elements: 3 Angular, 3 translational
    
    plant.SetPositions(plant_context,soccer_ball_model, np.array([1,0,0,0,0,params["s_ball_goalpost"], 0]))
    v, desired_pos = randomize_ball_trajectory_w_dp()
    vx, vy, vz = v
    xf, yf, zf = desired_pos
    plant.SetVelocities(plant_context,soccer_ball_model, np.array([0,0,0,vx,vy,vz]))

    #Calculate initial pose and intersection pose
    end_effector = plant.GetBodyByName('goalie_end_link')

    # Confirm that simulation works:
    simulator.AdvanceTo(.001)
    meshcat.Flush()
    if teleop:
        sliders.Run(diagram, default_interactive_timeout)
    
    if running_as_notebook:
        simulator.set_target_realtime_rate(1.0)
        visualizer.StartRecording(True)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        visualizer.PublishRecording()
        meshcat.DeleteButton("Stop Simulation")
        print("Simulation complete")

create_soccer_scene_base()


Press Escape to stop the simulation


KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c79dd6be-c791-4b96-b0a1-e37f965290d0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>